In [1]:
# https://github.com/pytorch/pytorch/blob/3cbe7a53a9a1cea2ef2a042f1ab6f7758f7e4d74/torch/csrc/api/include/torch/nn/functional/activation.h#L643

In [2]:
# https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [3]:
# https://youtu.be/kCc8FmEb1nY?si=AEN-_b8nxN1nxvDf&t=5248

In [ ]:
# https://github.com/karpathy/nanoGPT/issues/399

In [4]:
# sentencepiece
# tiktoken

In [5]:
# import tiktoken
# enc = tiktoken.get_encoding('gpt2')
# enc.encode('hi my name is linsu!')
# enc.decode([5303, 616, 1438, 318, 300, 1040, 84, 0])

![](diagram.png)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
B, T, E = 4, 8, 2
x = torch.randn(B, T, E)
xbow = torch.zeros((B, T, E))

In [8]:
# method 1
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, D)
        xbow[b, t] = torch.mean(xprev, 0) # (D,)

In [9]:
# method 2
wei = torch.tril(torch.ones((T, T)))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (-, T, T) @ (B, T, C) = (B, T, C)

In [10]:
# method 3
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x # (-, T, T) @ (B, T, C) = (B, T, C)

In [11]:
# method 4: Attention Mechanism
head_dim = 16
query = nn.Linear(E, head_dim, bias=False)
key = nn.Linear(E, head_dim, bias=False)
value = nn.Linear(E, head_dim, bias=False)
q = query(x) # q = Wx (B, T, 16)
k = key(x) # k = Wx (B, T, 16)
v = value(x) # v = Wx (B, T, 16)

# dot product between two vectors measures similarity
# this matrix multiplication dots each vector to every other vector
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (8, 16, T) = (B, T, T)
wei = wei * head_dim**(-0.5)

tril = torch.tril(torch.ones(T, T)) # optional (for decoder)
wei = wei.masked_fill(tril == 0, float('-inf')) # optional (for decoder)
wei = F.softmax(wei, dim=-1)
out = wei @ v # (B, T, T) @ (B, T, H) = (B, T, H)